In [1]:
import numpy as np
import os
import cv2
import time
import glob
import json
import pandas as pd
import io
import os.path
import matplotlib.pyplot as plt
from PIL import Image

from apiclient.http import MediaFileUpload, MediaIoBaseDownload
from genericpath import exists
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from bs4 import BeautifulSoup
import urllib.request as req
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from statistics import mode
import datetime


In [2]:
def raceGetNumber():
    lists = []
    for i in range(1, 13):
        time.sleep(1)
        
        html = 'https://nar.netkeiba.com/race/shutuba.html?race_id=2022541113{}'.format(str(i).zfill(2))

        data = pd.read_html(html)[0]

        hourseNumber = []
        table = []

        for j in range(13):
            table.append(data.columns[j][0])

        data.columns = table

        for j in data['馬番']:
            hourseNumber.append(j)

        lists.append(hourseNumber)
    return lists

In [3]:
def ocr(model, output_path):
    # 入力画像のパラメータ
    img_width = 32 # 入力画像の幅
    img_height = 32 # 入力画像の高さ
    img_ch = 3 # 3ch画像（RGB）

    # 入力データ数
    num_data = 1

    # 画像の読み込み（32×32にリサイズ）
    # 正規化, 4次元配列に変換（モデルの入力が4次元なので合わせる）
    img = load_img(output_path, target_size=(img_width, img_height))
    img = img_to_array(img) 
    img = img.astype('float32')/255.0
    img = np.array([img])

    # 分類機に入力データを与えて予測（出力：各クラスの予想確率）
    y_pred = model.predict(img,verbose=0)

    # 最も確率の高い要素番号
    number_pred = np.argmax(y_pred) 
    if 0.8 < y_pred[0][number_pred]:
        kd = 9
        # print('number_pred : ' + str(number_pred))
    else:
        number_pred = 99
        # print('number_pred : ' + str(number_pred))

    return number_pred

In [4]:
def kouchi_driver():
    options = Options() 
    # options.headless = True
    options.add_argument('--headless')                 # headlessモードを使用する
    options.add_argument('--disable-gpu')              # headlessモードで暫定的に必要なフラグ(そのうち不要になる)
    options.add_argument('--disable-extensions')       # すべての拡張機能を無効にする。ユーザースクリプトも無効にする
    options.add_argument('--proxy-server="direct://"') # Proxy経由ではなく直接接続する
    options.add_argument('--proxy-bypass-list=*')      # すべてのホスト名
    options.add_argument('--start-maximized')      
    driver = Chrome(executable_path="", options=options)
    # driver = Chrome(executable_path="")

    driver.get("https://simple.keiba-lv-st.jp/?track=kouchi")

    # 画質変更ボタンクリック
    elem = driver.find_element(By.XPATH,'/html/body/div/div/div[2]/div[1]/div[1]/div[2]/button')
    elem.click()
    # 2000p選択
    elem = driver.find_element(By.XPATH,'/html/body/div/div/div[2]/div[1]/div[1]/div[2]/ul/li[6]/a')
    elem.click()
    # # 拡大画面クリック
    elem = driver.find_element(By.XPATH,'//*[@id="player-controls-player_1"]/img[2]')
    elem.click()

    return driver

In [5]:
# 映像から馬の画像を抽出する
def taskRace(raceNum, hourseNumber):
    output_path = ""
    create_output_path = ""
    # 保存したモデル構造の読み込み
    model = model_from_json(open("", 'r').read())
    # 保存した学習済みの重みを読み込み
    model.load_weights("")

    print('パドック馬保存開始')
    driver = kouchi_driver()

    num = 0
    dateT = datetime.datetime.now()
    # 20分後の時刻
    dt2 = dateT + datetime.timedelta(minutes=25)
    old = datetime.time(dt2.hour, dt2.minute)

    while True:
        time.sleep(1)
        nowTime = datetime.datetime.now()
        new_ = datetime.time(nowTime.hour, nowTime.minute)

        if(old < new_):
            break
        
        ocrNumList = []

        img_png = driver.get_screenshot_as_png()
        img_io = io.BytesIO(img_png)
        img = Image.open(img_io)

        # OCR用
        ocrimg = img.crop((175, 370, 233, 420))
        ocrimg.save(output_path)
        
        # 保存用
        imgHorse = img.crop((55, 100, 720, 490))
        CreateOCRimg = cv2.imread(output_path)
        # 自作OCR適用
        for i in range(5):
            
            if(i == 0):
                ocrNum = ocr(model, output_path)
                ocrNumList.append(ocrNum)
            if(i == 1):
                black(CreateOCRimg,create_output_path)
                ocrNum = ocr(model, create_output_path)
                ocrNumList.append(ocrNum)
            if(i == 2):
                affine(CreateOCRimg,create_output_path)
                ocrNum = ocr(model, create_output_path)
                ocrNumList.append(ocrNum)
            if(i == 3):
                gamma_correction(CreateOCRimg,0.1,create_output_path)
                ocrNum = ocr(model, create_output_path)
                ocrNumList.append(ocrNum)
            if(i == 4):
                gamma_correction(CreateOCRimg,1.9,create_output_path)
                ocrNum = ocr(model, create_output_path)
                ocrNumList.append(ocrNum)
        
        # 一番出現回数の多い馬番号を取得
        ocrNum = mode(ocrNumList)
        print("OCR番号："+ str(ocrNum))
        if(str(ocrNum) == 99):
            continue       
        try:
            for hNum in hourseNumber:
                if str(hNum) == str(ocrNum):
                    print(str(num) + '頭目')
                    savepath = "".format(str(raceNum + 1))
                    savepath = savepath + str(hNum)
                    # フォルダがなかったら作成
                    if(True != (os.path.isdir(savepath))):
                        os.makedirs(savepath)
                        print("来てる")
                    imgHorse.save(savepath + "/horse_{}.png".format(num)) 
                    num += 1
                    break
        except Exception as e:
            print(e.args)
            continue
            

In [6]:
def black(img, path):
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,img_2chika = cv2.threshold(img_grayscale,200,255,cv2.THRESH_BINARY)
    cv2.imwrite(path, img_2chika)

def affine(img, path):
    rows,cols,ch = img.shape
    pts1 = np.float32([[0,50],[50,0],[10,50]])
    pts2 = np.float32([[0,50],[40,10],[10,50]])
    M = cv2.getAffineTransform(pts1,pts2)
    img_afin = cv2.warpAffine(img,M,(cols,rows))
    cv2.imwrite(path, img_afin)

def perspective(img, path):
    rows,cols,ch = img.shape
    pts1 = np.float32([[0,0],[100,0],[0,100],[100,100]])
    pts2 = np.float32([[20,0],[80,0],[0,100],[100,100]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    img_syaei = cv2.warpPerspective(img,M,(100,100))
    cv2.imwrite(path, img_syaei)

def gamma_correction(image, gamma, path):#輝度変化用の関数
    lookup_table = np.zeros((256, 1), dtype = 'uint8')  
    for loop in range(256):    
        lookup_table[loop][0] = 255 * pow(float(loop)/255, 1.0/gamma) 
    image_gamma = cv2.LUT(image, lookup_table) 

    cv2.imwrite(path, image_gamma)

In [ ]:
horseNumber = lists[7]
taskRace(7,horseNumber)

In [ ]:
import schedule
import time

# レース情報読み込み
lists = raceGetNumber()


# 実行job関数
def job(x):
    # レース番号
    horseNumber = lists[x]
    taskRace(x,horseNumber)


while True:
    schedule.run_pending()
    # schedule.every().day.at("15:00").do(job, x=1)
    # schedule.every().day.at("15:35").do(job, x=2)
    # schedule.every().day.at("16:10").do(job, x=3)
    # schedule.every().day.at("16:45").do(job, x=4)
    # schedule.every().day.at("17:26").do(job, x=5)
    schedule.every().day.at("17:55").do(job, x=6)
    schedule.every().day.at("18:25").do(job, x=7)
    schedule.every().day.at("18:55").do(job, x=8)
    schedule.every().day.at("19:25").do(job, x=9)
    schedule.every().day.at("19:55").do(job, x=10)
    schedule.every().day.at("20:25").do(job, x=11)
    time.sleep(57)